In [1]:
import re

tbankfiles = {
    'glaux1.0' : '/Users/gcrane/github/glaux-trees/public/xml',
    'glaux2.0' : '/Users/gcrane/github/glaux/xml'
}

curwork = '0032-006'

tbanks = {}
lemmaindex = {}
formindex = {}

def gettbank(tbankname,version):
    formindex = {}
    lemmaindex = {}
    index2sent = {}
    index2cit = {}
    
    subdoc = ''
    sentence = ''
    wordid = ''
    lemma = ''
    fname = tbankfiles['glaux1.0'] + '/' + curwork + '.xml'
    f = open(fname)
    for l in f:
        m = re.search('subdoc="([^"]+)"',l)
        if(m):
            subdoc = m[1]
        m = re.search('sentence id="([^"]+)"',l)
        if(m):
            sentence = m[1]
        m = re.search('word id="([^"]+)"',l)
        if(m):
            wordid = m[1]
        m = re.search('form="([^"]+)"',l)
        if(m):
            form = m[1]
            if(not form in formindex):
                formindex[form] = []
                
            formindex[form].append(wordid)
        m = re.search('lemma="([^"]+)"',l)
        if(m):
            lemma = m[1]
            if(not lemma in lemmaindex):
                lemmaindex[lemma] = []
            lemmaindex[lemma].append(wordid)
    f.close()
    return(lemmaindex,formindex)

lemmaindex[curwork], formindex[curwork] = gettbank(curwork,'glaux1.0')

In [3]:
import re



def docrosb(fname):
    inforgloss = 0
    f = open(fname)
    outf = ''
    incit = 0
    curlesson = ''
    workl = ''
    curhead = ''
    curlisttype = ''
    totwords = 0
    for l in f:
        l = re.sub('\s+$','',l)
        l = re.sub('<cell><rs type="lemma">([^<]+)</rs></cell><cell>([0-9]+)','<cell>[\g<1>](https://atlas-test.fly.dev/morphology/lemmas/?lang=grc&q=\g<1>)</cell><cell>[\g<2>](https://github.com/gregorycrane/CrosbySchaeffer2.0/tree/main/chaps/vocpassages/\g<1>.md)',l)

        l = re.sub('<figure><head>','<head>Figure: ',l)
        l = re.sub('</figure>','',l)
        
        l = re.sub('url="images/([^"]+)"','url="https://github.com/gregorycrane/CrosbySchaeffer2.0/blob/main/chaps/images/\g<1>?raw=true"',l)
        m = re.search('lesson" n="([0-9]+)"',l)
        
        if(m):
            if(outf):
                outf.close()
            outfname = 'chaps/lesson' + re.sub('^(.)$','0\g<1>',m[1]) + '.md'
            curlesson = m[1]
            curhead = ''
            print(outfname)
            #continue
            outf = open(outfname,'w')
            continue
        m = re.search('<item><rs type="form">([^<]+)</rs>',l)
        if(m):
            curform = m[1]
            print('curlem',curform)
            if(not curform in formindex[curwork]):
                lemfreq = 0
            else:
                lemfreq = len(formindex[curwork][curform])
            totwords = totwords + lemfreq
            newl = '| ' + curform + ' | ' + str(lemfreq) + ' | ' +str(totwords) + ' | '
            l = re.sub('<item><rs type="form">([^<]+)</rs>[:,\s]+(.+)</item>',newl + '\g<2>',l)

        m = re.search('<item><rs type="lemma">([^<]+)</rs>',l)
        if(m):
            curlemma = m[1]
            curlemma = re.sub('ᾱ','α',curlemma)
            print('curlem',curlemma)
            totwords = totwords + len(lemmaindex[curwork][curlemma])
            lemfreq = str(len(lemmaindex[curwork][curlemma]))
            newl = '| ' + curlemma + ' | [' + lemfreq + '](https://github.com/gregorycrane/CrosbySchaeffer2.0/tree/main/chaps/vocpassages/'+curlemma+'.md) | ' +str(totwords) + ' | '
            l = re.sub('<item><rs type="lemma">([^<]+)</rs>[:,\s]+(.+)</item>',newl + '\g<2>',l)
        m = re.search('<item><rs type="lemma" n="([^"]+)">([^<]+)</rs>',l)
        if(m):
            curlemma = m[1]
            uselemma = m[2]
            uselemma = re.sub('ᾱ','α',uselemma)
            curlemma = re.sub('ᾱ','α',curlemma)
            print('curlem',curlemma)
            totwords = totwords + len(lemmaindex[curwork][curlemma])
            lemfreq = str(len(lemmaindex[curwork][curlemma]))
            newl = '| ' + curlemma + ' | [' + lemfreq + '](https://github.com/gregorycrane/CrosbySchaeffer2.0/tree/main/chaps/vocpassages/'+uselemma+'.md) | ' +str(totwords) + ' | '
            l = re.sub('<item><rs type="lemma"[^>]*>([^<]+)</rs>[:,\s]+(.+)</item>',newl + '\g<2>',l)

            
        if(re.search('<cit>',l)):
            l = re.sub('<cit>','',l)
            incit = 1
        if(re.search('</cit>',l)):
            l = re.sub('</cit>','',l)
            incit = 0

        l = re.sub('<ref n="(http[^"]+)">(.+?)</ref>','[\g<2>](\g<1>)',l)
        l = re.sub('<figure><head>(.+)</head><p>(.+)</p></figure>','\n\n*\g<1>*: \g<2>\n',l)

        l = re.sub('^<head>(.+)</head>','# Chapter '+curlesson+': \g<1>',l)

        l = re.sub('<figure><graphic url="([^"]+)"/><head>([^<]+)</head></figure>','![text](\g<1> \g<2>)',l)
        l = re.sub('<figure><graphic url="([^"]+)"/></figure>','![text](\g<1>)',l)
        l = re.sub('<graphic url="([^"]+)"/>','![text](\g<1>)',l)
            
        if(re.search('<quote[^>]*>(.+)</quote>',l)):
            l = re.sub('<quote[^>]*>(.+)</quote>',' >  '+ '\g<1><br/>',l)
        if(re.search('<title[^>]*>(.+)</title>',l)):
            l = re.sub('<title[^>]*>(.+)</title>','*\g<1>*',l)
        m = re.search('\s*<bibl n="(http[^"]+)">([^<]+)</bibl>',l)
        if(m):
            l = re.sub('\s*<bibl n="(http[^"]+)">([^<]+)</bibl>','>> [\g<2>](\g<1>)',l)
        if(re.search('^\s*<bibl[^>]*>(.+)</bibl>',l)):
            l = re.sub('<bibl[^>]*>(.+)</bibl>','>> '+ '\g<1>',l)
            
        m = re.search('<div type="textpart" subtype="para" n="([0-9a-z]+)"><head>(.+)</head>',l)
        if(m):
            curpara = m[1]
            l = re.sub('<div type="textpart" subtype="para" n="([0-9a-z]+)"><head>(.+)</head>','\n## [§ \g<1>](#para\g<1>). \g<2>',l)
          
        l = re.sub('</p></div>','',l)
        
        if(re.search('<list type="vocab">',l)):
            curlisttype = 'vocab'
            l = re.sub('\s*<list type="vocab">','| Lemma | Freq | Total | Gloss |\n\r| --- | --- | --- | -- |',l)
            print('| Lemma | Freq | Total | Gloss |',file=outf)
            print('| --- | --- | --- | -- |',file=outf)
            continue
        if(re.search('</list>',l)):
            curlisttype = ''
        
        m = re.search('<list><head>(.+)</head',l)
        
        if(m):
            inlist = 'plain'
        
            l = re.sub('<list><head>(.+)</head>','### \g<1>\n',l)
            
        if(re.search('</list>',l)):
            l = re.sub('</list>','',l)
        l = re.sub('<item>(.+)</item>','- \g<1>',l)
            
        m = re.search('<item n=[\'"]([0-9]+)[\'"]>',l)
        if(m):
            listn = m[1]
            l = re.sub('<item n=[\'"]([0-9]+)[\'"]>','\g<1>. ',l)
            
        l = re.sub('</item>','',l)
        
        l  = re.sub('(<quote[^>]*><l>)',' > ',l)
        l  = re.sub('(<l>)',' >> ',l)
        l  = re.sub('</l>','',l)
        l = re.sub('</quote>','',l)            
        l = re.sub('<emph>(.+?)</emph>','**\g<1>**',l)
                  
        l = re.sub('(<table>|</table>)','\n',l)
        if(re.search('<row role="label"',l)):
            workl = re.sub('<cell>[^<]*</cell>','| --- ',l)
            workl = re.sub('^\s+','',workl)
            workl = re.sub('(<row[^>]*>|</row>)','',workl)
            #print('workl',l)
        else:
            workl = ''
        l = re.sub('<cell>([^<]*)</cell>','| \g<1> | ',l)
        l = re.sub('(<row[^>]*>|</row>)','',l)
        l = re.sub('\|\s*\|','|',l)
        l = re.sub('^\s+','',l)
        
        l = re.sub('(</p>|</div>)','',l)
        
        l = re.sub('<term>([^>]+)</term>','**\g<1>**',l)
            
        m = re.search('^\s*<foreign>(.+?)</foreign>\s*<gloss>(.+?)</gloss>\s*$',l)
        if(m):
            if(not inforgloss):
                print('| Greek | English | ',file=outf)
                print('| --- | -- | ',file=outf)
            print('| ',m[1], ' | ', m[2], ' |',file=outf)
            inforgloss= 1
            continue
        l = re.sub('(<list>|</list>)','',l)
        l = re.sub('\s*<item>','- ',l)
        l = re.sub('\s*(Note:)','> **Note**:',l)
        l = re.sub('<p>','\n\n',l)
        
        l = re.sub('<foreign[^>]+>([^<]+)</foreign>([\s,]+)<gloss>([^<]+)</gloss>','"\g<1>"\g<2>*\g<3>*',l)
        l = re.sub('<gloss>([^<]+)</gloss>','*\g<1>*',l)

        if(outf):
            inforgloss = 0
            print(l,file=outf)
            if(workl):
                print(workl,file=outf)
                workl = ''
    
    f.close()
    
docrosb('CrosbySchaefer2.1.xml')

chaps/lesson00.md
chaps/lesson01.md
curlem ἀδελφός
curlem ἔχει
curlem ἔχουσι
curlem ἦν
curlem ἦσαν
curlem ὁ
curlem παύει
curlem παύουσι
curlem πέμπει
curlem πέμπουσι
curlem ποταμός
curlem στρατηγός
chaps/lesson02.md
curlem ἄνθρωπος
curlem ἄξιος
curlem ἀπό
curlem δίκαιος
curlem Ἑλλήσποντος
curlem ἐκ
curlem μακρός
curlem μικρός
curlem πόλεμος
curlem πολέμιος
curlem φίλος
chaps/lesson03.md
curlem ἀγαθός
curlem ἄγγελος
curlem ἄγω
curlem γράφω
curlem ἐθέλω
curlem ἵππος
curlem καί
curlem καλός
curlem κίνδυνος
curlem λίθος
curlem λύω
curlem παύω
chaps/lesson04.md
curlem δέ
curlem ἐν
curlem δένδρον
curlem παρά
curlem δῆλος
curlem δῶρον
curlem εἰς
curlem πεδίον
curlem στάδιον
chaps/lesson05.md
chaps/lesson06.md
curlem γάρ
curlem σπονδή
curlem κώμη
curlem μάχη
curlem φεύγω
curlem οὐ
curlem φυγή
curlem φυλακή
curlem σκηνή
curlem φυλάσσω
chaps/lesson07.md
curlem ἀγορά
curlem ἀρχή
curlem δέκα
curlem διά
curlem ἐπιτήδειος
curlem ἡμέρα
curlem στρατιά
curlem φιλία
curlem φίλιος
curlem χώρα
chaps/lesso

KeyError: 'σῑγή'